# Importing Libraries

In [31]:
import cv2
import tensorflow as tf
from keras.models import load_model
import numpy as np
from pygame import mixer

from google.colab.patches import cv2_imshow
from IPython.display import clear_output

In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model('/content/drive/MyDrive/AI Project/models/Inception_model.h5')


In [ ]:
video = '/content/drive/MyDrive/AI Project/Test Videos/video1.mp4'


In [ ]:
mixer.init()
sound = mixer.sound(r'C:\Users\Justice Ntoi\PycharmProjects\AI-Drowsiness-Detection-Model\alarm\mixkit-alarm-990.wav')

cap = cv2.VideoCapture(video)

counter = 0

while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors=1)

    cv2.rectangle(frame, (0,height-50), (200,height), (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)

    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(frame, pt1=(ex, ey), pt2=(ex+ew, ey+eh), color=(0, 255, 0), thickness=2)
        # Preprocessing
        eye = frame[ey: ey+eh, ex: ex+ew]
        eye = cv2.resize(eye)
        eye = eye/255
        eye = eye.reshape(80,80,3)
        eye = np.expand_dims(eye, axis=0)

        # Prediction
        prediction = model.predict(eye)

        # Open Eyes
        if prediction[0][0]>0.90:
            cv2.putText(frame, 'Open', (10,height-20), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=1, color=(255,255,255), thickness=2, lineType=cv2.LINE_4)

        # Closed Eyes
        elif prediction[0][1]>0.30:
            cv2.putText(frame, 'Closed', (10,height-20), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=1, color=(255,255,255), thickness=2, lineType=cv2.LINE_4)

            counter = counter + 1
            if counter>10:
                try:
                    sound.play()
                except:
                    pass
                counter = 0
    # _________________________________________________________________________________________

    clear_output(wait=True)
    cv2_imshow(frame)

    # __________________________________________________________________________________________

    # cv2.imshow('frame', frame)
    if cv2.waitKey(2) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()